In [ ]:
# Install required packages

!pip install numpy
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126
!pip install sentence-transformers
!pip install faiss-cpu
!pip install datasets
!pip install pillow
!pip install librosa
!pip install --no-deps hazm
!pip install fasttext-wheel flashtext gensim
!pip install gradio
!pip install bitsandbytes
!pip install arabic-reshaper python-bidi
!pip install python-crfsuite
!pip install -q sentence-transformers huggingface_hub
!pip install trl
!pip install peft
!pip install bitsandbytes
!pip install accelerate
!pip install thefuzz
!pip install nltk

# Verify GPU availability
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

In [22]:
# print(torch.cuda.memory_summary())
# # del model
# # del optimizer
# # del loss
# # del outputs
# import gc, torch
# gc.collect()
# torch.cuda.empty_cache()

# torch.cuda.reset_peak_memory_stats()
# torch.cuda.reset_accumulated_memory_stats()



In [ ]:
!pip install tqdm
!pip install matplotlib

In [24]:
# For API libraries
import base64
import mimetypes
from huggingface_hub import InferenceClient
import os

In [25]:
import os
import json
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob  # For finding all JSON files

# Set project path
INPUT_PROJECT_PATH = r"E:\project\pasta\NLP_Project"
PROJECT_PATH = r"E:\project\pasta"

os.makedirs(PROJECT_PATH, exist_ok=True)
os.chdir(PROJECT_PATH)


In [26]:
print(INPUT_PROJECT_PATH)
print(PROJECT_PATH)

E:\project\pasta\NLP_Project
E:\project\pasta


In [27]:
DATA_PATH = os.path.join(INPUT_PROJECT_PATH, "Data")## update this later
IMAGE_BASE_PATH = os.path.join(DATA_PATH, "images")
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(IMAGE_BASE_PATH, exist_ok=True)
print(DATA_PATH)
print(IMAGE_BASE_PATH)

E:\project\pasta\NLP_Project\Data
E:\project\pasta\NLP_Project\Data\images


In [ ]:
for (root, dirs, files) in os.walk("." ,topdown= True, followlinks =True):
    print(f"\nIn folder: {root},{dirs},{files}")
    for file in files:
        print(f"  - {file}")


In [29]:


# json_files = glob.glob(os.path.join(DATA_PATH, '**', '*.json'), recursive=True)

# # json_files = glob.glob(os.path.join(DATA_PATH, '*.json'))
# # json_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.json')]
# json_files.sort()
# print(f"Found {len(json_files)} JSON files:")
# for f in json_files:
#     print(f)



# Import libraries with Persian support
import hazm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    CLIPProcessor,
    CLIPModel,
    pipeline
)
import faiss

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


In [30]:
import os
import json
import glob
from tqdm.notebook import tqdm # Using the notebook version for a cleaner progress bar

import os

def correct_path(path_from_json: str, base_data_path: str) -> str:
    """
    Cleans a broken path (e.g., from Windows with absolute paths) and reconstructs
    it based on a correct base path for the current environment.

    Args:
        path_from_json (str): The potentially broken path string from the JSON file.
        base_data_path (str): The correct, absolute base path to your data folder.

    Returns:
        str: A corrected, absolute path for the current OS, or an empty string if input is invalid.
    """
    if not isinstance(path_from_json, str) or not path_from_json:
        return ""

    # 1. Standardize all slashes to forward slashes for consistent processing.
    normalized_path = path_from_json.replace('\\', '/')

    # 2. Intelligently find the start of the relative path by looking for key folders.
    key_folders = ['images/', 'downloaded_images/']
    relative_path = None

    for key_folder in key_folders:
        if key_folder in normalized_path:
            # Split the path at the key folder and take everything that comes after it.
            # Then, rejoin the key folder to the start of this relative part.
            relative_part = normalized_path.split(key_folder, 1)[1]
            relative_path = os.path.join(key_folder.strip('/'), relative_part)
            break
    
    # 3. If no key folder was found, fall back to just using the filename.
    if relative_path is None:
        relative_path = os.path.basename(normalized_path)

    # 4. Join the correct base path with the extracted relative path.
    return os.path.join(base_data_path, relative_path)


# Initialize data storage
all_data = []

# --- DATA LOADING ---
json_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.json')]
json_files.sort()

if not json_files:
    print(f"No JSON files found in '{DATA_PATH}'. Please check the directory.")
else:
    print(f"Found {len(json_files)} JSON files: {json_files}")
    for file_name in json_files:
        file_path = os.path.join(DATA_PATH, file_name)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data_from_file = json.load(f)

                # Fix image paths by making them absolute and handling string/list inconsistencies
                for item in data_from_file:
                    image_paths_raw = item.get("images", [])

                    # If image_paths_raw is a string, put it in a list
                    if isinstance(image_paths_raw, str):
                        image_paths_raw = [image_paths_raw]

                    # Now, image_paths_raw is guaranteed to be a list.
                    # Create full, absolute paths for each image.
                    item["images"] = [os.path.join(DATA_PATH, img_path.strip()) for img_path in image_paths_raw]

                all_data.extend(data_from_file)
                print(f"✓ Successfully loaded {len(data_from_file)} entries from {file_name}")
        except Exception as e:
            print(f"✗ Error reading {file_name}: {e}")

    print(f"\nTotal items loaded from all files: {len(all_data)}")
    if all_data:
        print("\nSample of the first loaded item:")
        print(json.dumps(all_data[0], indent=2, ensure_ascii=False))

        # --- VERIFICATION AND CORRECTION FUNCTION ---
        def verify_and_correct_image_paths(data_list):
            """
            Verifies image paths. If an extension is missing, it searches for a matching file
            (e.g., '.jpg', '.png') and corrects the path in the data list.
            """
            print("\nVerifying and correcting image paths...")
            total_found = 0
            total_expected = 0
            missing_paths = []

            for i, item in enumerate(tqdm(data_list, desc="Checking images")):
                if "images" not in item:
                    continue

                image_paths_list = item.get("images", [])
                total_expected += len(image_paths_list)

                corrected_paths = []
                for path in image_paths_list:
                    # Check if the file exists as is
                    if os.path.isfile(path):
                        corrected_paths.append(path)
                        total_found += 1
                    else:
                        # If not, try to find it by adding a wildcard
                        search_pattern = f"{path}.jpg"
                        found_files = glob.glob(search_pattern)

                        if found_files:
                            # Found a match! Use the first one found.
                            full_path = found_files[0]
                            corrected_paths.append(full_path)
                            total_found += 1
                        else:
                            path = correct_path(path, DATA_PATH)
                            # Still not found, it's genuinely missing.
                            missing_paths.append(path)

                # CRITICAL: Update the original data with the corrected paths
                data_list[i]["images"] = corrected_paths

            if total_expected > 0:
                print(f"\nOverall: Found {total_found} out of {total_expected} expected images ({total_found/total_expected:.1%})")

            if missing_paths:
                print("\n--- The following image paths could not be found: ---")
                for path in missing_paths:
                    print(path)
                print("----------------------------------------------------")
                print("Please check for typos or ensure these files are in the correct location in your Google Drive.")
            return total_found > 0

        # Run the verification and correction
        if verify_and_correct_image_paths(all_data):
            tourism_data = all_data
            print("\nTourism data successfully loaded and verified!")
        else:
            print("\nWarning: Some or all images were not found. Please check the paths printed above.")
            tourism_data = all_data

    else:
        print("No valid data loaded from JSON files.")
        tourism_data = []


Found 18 JSON files: ['data_Ardabil.json', 'data_Azar_Q.json', 'data_Azar_S.json', 'data_Bushehr.json', 'data_Chaharmahal_and_Bakhtiari.json', 'data_Fars.json', 'data_Gilan.json', 'data_Hamedan.json', 'data_Hormozgan.json', 'data_Ilam.json', 'data_Isfahan.json', 'data_Kerman.json', 'data_Kermanshah.json', 'data_Khuzestan.json', 'data_Kohgiluyeh_V2.json', 'data_Kord_V2.json', 'data_Yazd.json', 'data_Zanjan_V2.json']
✓ Successfully loaded 80 entries from data_Ardabil.json
✓ Successfully loaded 92 entries from data_Azar_Q.json
✓ Successfully loaded 156 entries from data_Azar_S.json
✓ Successfully loaded 7 entries from data_Bushehr.json
✓ Successfully loaded 54 entries from data_Chaharmahal_and_Bakhtiari.json
✓ Successfully loaded 15 entries from data_Fars.json
✓ Successfully loaded 76 entries from data_Gilan.json
✓ Successfully loaded 59 entries from data_Hamedan.json
✓ Successfully loaded 134 entries from data_Hormozgan.json
✓ Successfully loaded 48 entries from data_Ilam.json
✓ Successf

Checking images:   0%|          | 0/1079 [00:00<?, ?it/s]


Overall: Found 4552 out of 4552 expected images (100.0%)

Tourism data successfully loaded and verified!


In [ ]:
import os
import json
import glob
from tqdm.notebook import tqdm # Using the notebook version for a cleaner progress bar

# Initialize data storage
all_data = []
print("Files in DATA_PATH:", os.listdir(DATA_PATH))
json_files = glob.glob(os.path.join(DATA_PATH, '**', '*.json'), recursive=True)

# json_files = glob.glob(os.path.join(DATA_PATH, '*.json'))
# json_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.json')]
json_files.sort()
print(f"Found {len(json_files)} JSON files:")
for f in json_files:
    print(f)

for root, dirs, files in os.walk(DATA_PATH):
    print(f"\nIn folder: {root}")
    for file in files:
        print(f"  - {file}")


if not json_files:
    print(f"No JSON files found in '{DATA_PATH}'. Please check the directory.")
else:
    print(f"Found {len(json_files)} JSON files: {json_files}")
    for file_name in json_files:
        file_path = os.path.join(DATA_PATH, file_name)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data_from_file = json.load(f)

                # Fix image paths: prepend the base DATA_PATH to make them absolute
                for item in data_from_file:
                    item["images"] = [os.path.join(DATA_PATH, img) for img in item.get("images", [])]

                all_data.extend(data_from_file)
                print(f"✓ Successfully loaded {len(data_from_file)} entries from {file_name}")
        except Exception as e:
            print(f"✗ Error reading {file_name}: {e}")

    print(f"\nTotal items loaded from all files: {len(all_data)}")
    if all_data:
        print("\nSample of the first loaded item:")
        print(json.dumps(all_data[0], indent=2, ensure_ascii=False))

        # --- UPDATED VERIFICATION FUNCTION ---
        def verify_image_paths(data):
            """
            Verify that image paths exist and create a list of any that are missing.
            """
            print("\nVerifying image paths...")
            total_found = 0
            total_expected = 0
            missing_files = []  # List to store the paths of missing files

            # Use tqdm for a nice progress bar
            for item in tqdm(data, desc="Checking images"):
                image_paths = item.get("images", [])
                total_expected += len(image_paths)

                for img_path in image_paths:
                    if os.path.exists(img_path):
                        total_found += 1
                    else:
                        # If the file doesn't exist, add its path to our list
                        missing_files.append(img_path)

            if total_expected > 0:
                print(f"\nOverall: Found {total_found} out of {total_expected} expected images ({total_found/total_expected:.1%})")

            # If any files were missing, print the list
            if missing_files:
                print("\n--- The following image paths could not be found: ---")
                for path in missing_files:
                    print(path)
                print("----------------------------------------------------")
                print("Please check for typos or ensure these files are in the correct location in your Google Drive.")

            return total_found > 0

        # Run the verification
        if verify_image_paths(all_data):
            tourism_data = all_data
            print("\nTourism data successfully loaded and verified!")
        else:
            # This message now appears only if verification runs but finds nothing
            print("\nWarning: Some or all images were not found. Please check the paths printed above.")
            tourism_data = all_data # Still assign the data, as the system can handle missing images
    else:
        print("No valid data loaded from JSON files.")
        tourism_data = []

In [32]:
# @title  Load NEW Fine-Tuned Models(Text and CLIP)
from sentence_transformers import SentenceTransformer

print("Loading the newly fine-tuned text model for the RAG system...")

# Update the model path to point to your new version
hw2_model_path = "E:\\project\\pasta\\NLP_HW3\\text_retriever_contrastive_v2"

try:
    # Load the new model
    text_retriever_model = SentenceTransformer(hw2_model_path, device=device)
    print(" Successfully loaded the contrastive-tuned text model (v2).")
except Exception as e:
    print(f" ERROR: Could not load the new model. Please check the path. Error: {e}")


# Load fine-tuned CLIP model for multimodal embeddings
print("Loading CLIP model for multimodal embeddings...")
fine_tuned_model_path = "E:\\project\\pasta\\NLP_HW3\\Finetuned_Clip_Model"
print(f"Loading specialized fine-tuned model from: {fine_tuned_model_path}")

clip_model = CLIPModel.from_pretrained(fine_tuned_model_path).to(device)
clip_processor = CLIPProcessor.from_pretrained(fine_tuned_model_path)
print(" Fine-tuned CLIP model loaded successfully!")


# --- 2. Initialize Persian NLP tools (This part stays the same) ---
print("\nSetting up Persian language processing...")
normalizer = hazm.Normalizer()
stemmer = hazm.Stemmer()
lemmatizer = hazm.Lemmatizer()

def preprocess_persian(text):
    if not text: return ""
    return normalizer.normalize(text)

# --- 3. Final Status ---
print("\nCore models loaded successfully!")



Loading the newly fine-tuned text model for the RAG system...
 Successfully loaded the contrastive-tuned text model (v2).
Loading CLIP model for multimodal embeddings...
Loading specialized fine-tuned model from: E:\project\pasta\NLP_HW3\Finetuned_Clip_Model


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


 Fine-tuned CLIP model loaded successfully!

Setting up Persian language processing...

Core models loaded successfully!


In [33]:
import os

def get_all_image_paths(location_data):
    """
    Gets all available image paths for a location directly from the
    pre-loaded 'images' key in the data.
    """

    image_paths = location_data.get("images", [])

    return image_paths if image_paths else None

def get_image_path(location_data, index=0):
    """
    Gets a single image path by its index from the list of available images.
    """
    # First, get all the available paths using the function above.
    all_paths = get_all_image_paths(location_data)

    # Check if the list of paths exists and if the requested index is valid.
    if all_paths and index < len(all_paths):
        return all_paths[index]

    # If no images are found or the index is out of bounds, return None.
    return None

def get_persian_embeddings(texts):
    """
    Generate embeddings for Persian text inputs using CLIP text encoder
    """
    # Filter out empty texts
    texts = [t for t in texts if t and t.strip()]
    if not texts:
        return None

    # Preprocess Persian text
    processed_texts = [preprocess_persian(text) for text in texts]

    inputs = clip_processor(text=processed_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_embeddings = clip_model.get_text_features(**inputs)

    # Normalize embeddings
    text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)
    return text_embeddings.cpu().numpy()

def get_image_embeddings(image_paths):
    """
    Generate embeddings for image inputs using CLIP image encoder
    """
    valid_paths = [p for p in image_paths if p and os.path.exists(p)]
    if not valid_paths:
        return None, None

    images = []
    for path in valid_paths:
        try:
            img = Image.open(path).convert("RGB")
            images.append(img)
        except Exception as e:
            print(f"Error loading image {path}: {str(e)}")

    if not images:
        return None, None

    inputs = clip_processor(images=images, return_tensors="pt").to(device)

    with torch.no_grad():
        image_embeddings = clip_model.get_image_features(**inputs)

    # Normalize embeddings
    image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
    return image_embeddings.cpu().numpy(), valid_paths

In [34]:
# Cell 3.5: Clear GPU Memory
import torch
import gc


# Force Python's garbage collector to run
gc.collect()

# Empty the GPU cache
torch.cuda.empty_cache()

print(" GPU Memory Cleared")

 GPU Memory Cleared


In [35]:
import faiss
import numpy as np
import os
import json

def build_text_only_index(data, model):
    """
    Creates and saves a FAISS index using the text descriptions.
    If a saved index exists, it loads it instead of rebuilding.
    """
    save_path = "E:\\project\\pasta\\text_vector_db"
    index_file = os.path.join(save_path, "text_db.index")

    # 1. Check if a saved index already exists
    if os.path.exists(index_file):
        print("Found saved text vector database. Loading it...")
        try:
            text_index = faiss.read_index(index_file)
            print(f" Successfully loaded {text_index.ntotal} text vectors.")
            return text_index
        except Exception as e:
            print(f" Error loading text index file: {e}. Rebuilding from scratch.")

    # 2. If no saved index, build it from scratch
    print("Building text-only FAISS index from scratch...")
    os.makedirs(save_path, exist_ok=True)

    # Combine name and description for each location
    location_texts = [f"{loc.get('name', '')}: {loc.get('description', '')}" for loc in data]

    print(f"Generating embeddings for {len(location_texts)} text documents with the HW2 model...")

    # Generate embeddings using the SentenceTransformer model
    text_embeddings = model.encode(location_texts, convert_to_tensor=False, show_progress_bar=True)

    # IMPORTANT: Normalize the embeddings before adding to an Inner Product index
    faiss.normalize_L2(text_embeddings)

    embedding_dim = text_embeddings.shape[1]
    text_index = faiss.IndexFlatIP(embedding_dim) # Using Inner Product for cosine similarity
    text_index.add(np.array(text_embeddings, dtype='float32'))

    print(f"Text-only FAISS index created with {text_index.ntotal} vectors.")

    # 3. Save the newly created index to your Drive
    faiss.write_index(text_index, index_file)
    print(f" Text vector database saved to: {save_path}")

    return text_index


text_only_faiss_index = build_text_only_index(tourism_data, text_retriever_model)

Building text-only FAISS index from scratch...
Generating embeddings for 1079 text documents with the HW2 model...


Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Text-only FAISS index created with 1079 vectors.
 Text vector database saved to: E:\project\pasta\text_vector_db


In [ ]:
import faiss
import json
import os
from tqdm.notebook import tqdm
import numpy as np

def build_image_only_index(data, model, processor):
    """
    Creates a FAISS index using only the images and the CLIP model.
    """
    save_path = "E:\\project\\pasta\\image_vector_db"
    index_file = os.path.join(save_path, "image_db.index")
    metadata_file = os.path.join(save_path, "image_db_metadata.json")

    if os.path.exists(index_file) and os.path.exists(metadata_file):
        print("Found saved image database files. Loading them...")
        try:
            image_index = faiss.read_index(index_file)
            with open(metadata_file, 'r', encoding='utf-8') as f:
                image_metadata = json.load(f)
            print(f" Successfully loaded {image_index.ntotal} image vectors and metadata.")
            return image_index, image_metadata
        except Exception as e:
            print(f" Error loading image files: {e}. Rebuilding from scratch.")

    print("Building image-only FAISS index from scratch...")
    embedding_dim = 512  # CLIP image embedding dimension
    image_index = faiss.IndexFlatL2(embedding_dim)
    image_metadata = []

    for loc_idx, location in enumerate(tqdm(data)):
        image_paths = get_all_image_paths(location)
        if image_paths:
            img_embeddings, valid_paths = get_image_embeddings(image_paths)
            if img_embeddings is not None:
                for i, emb in enumerate(img_embeddings):
                    image_index.add(np.array([emb]))
                    image_metadata.append({
                        "loc_idx": loc_idx,
                        "path": valid_paths[i]
                    })

    print(f"Image-only FAISS index created with {image_index.ntotal} vectors.")
    os.makedirs(save_path, exist_ok=True)
    faiss.write_index(image_index, index_file)
    with open(metadata_file, 'w', encoding='utf-8') as f:
        json.dump(image_metadata, f, ensure_ascii=False, indent=4)
    print(f"Image database saved to: {save_path}")

    return image_index, image_metadata

# Add this line to build the new image index
image_only_faiss_index, image_metadata = build_image_only_index(tourism_data, clip_model, clip_processor)

In [37]:
# @title Create SURf Dataset

import json
import numpy as np
from tqdm.notebook import tqdm
import os
from thefuzz import fuzz

# --- Configuration ---
K_NEIGHBORS = 10
OUTPUT_FILENAME = "surf_training_data.json"
SIMILARITY_THRESHOLD = 50 # A score of 50% or higher means the names are too similar

def are_names_similar(name1, name2, threshold):
    """Checks if two names are similar using fuzzy matching."""
    # partial_ratio is good at finding a substring in another string, even with typos.

    similarity_score = fuzz.partial_ratio(name1, name2)
    return similarity_score >= threshold

def create_surf_dataset_final(data, retriever_model, faiss_index):
    print("Starting SURf dataset generation")
    surf_dataset = []

    for i in tqdm(range(len(data)), desc="Processing locations"):
        target_location = data[i]

        if not all(k in target_location for k in ['name', 'description', 'images']): continue
        positive_doc, target_name = target_location['description'], target_location['name'].strip()
        question = f"درباره {target_name} به من بگو"

        image_path = next((img for img in target_location['images'] if os.path.exists(img)), None)
        if not image_path: continue

        try:
            positive_embedding = retriever_model.encode([positive_doc], convert_to_numpy=True)
            distances, indices = faiss_index.search(positive_embedding, K_NEIGHBORS)

            negative_doc = None
            for j in range(1, len(indices[0])):
                retrieved_index = indices[0][j]
                retrieved_location = data[retrieved_index]

                if not all(k in retrieved_location for k in ['name', 'description']): continue
                retrieved_name = retrieved_location['name'].strip()

                # --- THE FINAL, ROBUST FUZZY CHECK ---
                if not are_names_similar(target_name, retrieved_name, SIMILARITY_THRESHOLD):
                    negative_doc = retrieved_location['description']
                    break

            if negative_doc is None: continue

        except Exception as e:
            continue

        training_example = {
            "image": image_path, "question": question, "positive_doc": positive_doc,
            "negative_doc": negative_doc, "ground_truth_answer": positive_doc
        }
        surf_dataset.append(training_example)

    return surf_dataset

# --- Main Execution ---
if 'all_data' in locals() and 'text_retriever_model' in locals() and 'text_only_faiss_index' in locals():
    generated_dataset = create_surf_dataset_final(all_data, text_retriever_model, text_only_faiss_index)
    output_path = os.path.join(PROJECT_PATH, OUTPUT_FILENAME)

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(generated_dataset, f, ensure_ascii=False, indent=4)

    print(f"\n Successfully created {len(generated_dataset)} training examples.")
    print(f"   Dataset saved to: {output_path}")
    if generated_dataset:
        print("\n--- Sample of the first training example ---")
        print(json.dumps(generated_dataset[0], indent=2, ensure_ascii=False))
else:
    print(" Please make sure you have run the previous cells to load the data, model, and FAISS index.")

Starting SURf dataset generation


Processing locations:   0%|          | 0/1079 [00:00<?, ?it/s]


 Successfully created 808 training examples.
   Dataset saved to: E:\project\pasta\surf_training_data.json

--- Sample of the first training example ---
{
  "image": "E:\\project\\pasta\\NLP_Project\\Data\\images/Ardabil_imgs/Lake_Shorabil_1.jpg",
  "question": "درباره دریاچه شورابیل به من بگو",
  "positive_doc": "اگر از مردم اردبیل بخواهید که چند مکان جذاب در این استان را به شما پیشنهاد دهند، مطمئنا یکی از گزینه‌هایی که آنها مطرح خواهند کرد دریاچه شورابیل است",
  "negative_doc": "سازه های آبی شوشتر از معروف‌ترین جاذبه‌های تاریخی در شهرستان شوشتر خوزستان است که در مجاورت بخش تاریخی این شهر قرار دارد. از سازه‌های آبی شوشتر با عنوان بزرگ‌ترین مجموعه صنعتی ایران باستان یاد می‌کنند و گردشگران خارجی بسیاری در سال‌های دور پس از بازدید از این سازه‌های تاریخی، آن‌ها را به جهان معرفی کردند. ساختار سازه‌های آبی شوشتر در زمان ساخت این مجموعه یکی از عجایب مهندسی در ایران باستان به شمار می‌رود. عملکرد این سازه‌ها نشان‌دهنده نبوغ و دانش بالای مهندسان ایران باستان نیز هستند. این سازه‌ها با بهره‌بردا

In [38]:
!pip install huggingface_hub[hf_xet]
!pip install hf_xet


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
# @title Fine-Tuning Qwen (SURf)
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForVision2Seq,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model
import os

os.environ["TORCH_HOME"] = r"E:\\torch"
os.environ["HF_HOME"] = r"E:\\huggingface"

# --- 1. Configuration ---
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
dataset_path = os.path.join(PROJECT_PATH, "surf_training_data.json")
new_model_name = "qwen-vl-7b-surf-finetuned-persian"
output_dir = os.path.join(PROJECT_PATH, new_model_name)

# --- 2. Load the Dataset ---
print(f"Loading dataset from: {dataset_path}")
dataset = load_dataset("json", data_files=dataset_path, split="train")

# Take one half (randomly selected)
# split_dataset = dataset.train_test_split(test_size=0.50, seed=42)
# dataset = split_dataset["train"] 


# --- 3. Define the Prompt Template ---
def format_prompt(example):
    system_prompt = (
        "شما یک دستیار هوشمند برای یک چت‌بات گردشگری هستید. "
        "وظیفه شما پاسخ به سوالات در مورد مکانی است که در تصویر نمایش داده شده است. "
        "دو قطعه متن به عنوان زمینه به شما داده می‌شود. یکی از آنها کاملاً مرتبط و صحیح است و دیگری گمراه‌کننده یا نامرتبط است. "
        "شما باید به طور انتخابی فقط از زمینه صحیح برای تولید پاسخ خود استفاده کنید. زمینه نامرتبط را کاملاً نادیده بگیرید."
    )
    prompt = f"""<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
[تصویر ضمیمه شده]

این هم زمینه:
[زمینه ۱ - مرتبط]: {example['positive_doc']}
[زمینه ۲ - نامرتبط]: {example['negative_doc']}

سوال: {example['question']}<|im_end|>
<|im_start|>assistant
{example['ground_truth_answer']}<|im_end|>"""
    return {"text": prompt}

dataset = dataset.map(format_prompt)

# --- 4. Configure Model and Tokenizer ---
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
model = AutoModelForVision2Seq.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# --- 5. Manually Tokenize the Dataset ---
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)#max length can be bigger

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=list(dataset.features))

# --- 6. Configure LoRA ---
lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
# Apply LoRA wrappers to the model
model = get_peft_model(model, lora_config)

# --- 7. Set Up Training Arguments ---
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=2,#this can be 2
    gradient_accumulation_steps=8,#this can be 4
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",#this can be removed
    save_strategy="epoch",
    logging_steps=10,
    report_to="none",
)

# --- 8. Initialize the Standard Trainer ---
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# --- 9. Start Training ---
print("\n--- Starting model fine-tuning with the standard Trainer... ---")
trainer.train()
print("--- Fine-tuning complete! ---")

# --- 10. Save the Model ---
print(f"Saving LoRA adapters to: {output_dir}")
trainer.save_model(output_dir)
print(" Model saved successfully.")

Loading dataset from: E:\project\pasta\surf_training_data.json


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/808 [00:00<?, ? examples/s]

e:\project\pasta\.venv_hazm\Lib\site-packages\transformers\models\auto\modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Map:   0%|          | 0/808 [00:00<?, ? examples/s]

C:\Users\nlp\AppData\Local\Temp\ipykernel_2064\2807693676.py:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.



--- Starting model fine-tuning with the standard Trainer... ---


Step,Training Loss
10,1.538400
20,0.433800
30,0.132100
40,0.121000
50,0.114400


--- Fine-tuning complete! ---
Saving LoRA adapters to: E:\project\pasta\qwen-vl-7b-surf-finetuned-persian
 Model saved successfully.


In [40]:
# @title Load the Fine-Tuned SURf Model
import torch
from peft import PeftModel
from transformers import (
    AutoModelForVision2Seq,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoProcessor
)
from PIL import Image
import os

# --- 1. Configuration ---
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
# Path to your new adapters in your OUTPUT folder
lora_adapter_path = os.path.join(PROJECT_PATH, "qwen-vl-7b-surf-finetuned-persian")

# --- 2. Load Base Model and Processor ---
print("Loading base model and processor...")
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
base_model = AutoModelForVision2Seq.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)
qwen_processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

# --- 3. Load and Merge LoRA Adapters ---
print(f"Loading LoRA adapters from: {lora_adapter_path}")
qwen_model = PeftModel.from_pretrained(base_model, lora_adapter_path)
qwen_model = qwen_model.merge_and_unload() # Merge for faster inference

print(" Fine-tuned SURf model is loaded and ready!")

Loading base model and processor...


e:\project\pasta\.venv_hazm\Lib\site-packages\transformers\models\auto\modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Loading LoRA adapters from: E:\project\pasta\qwen-vl-7b-surf-finetuned-persian


e:\project\pasta\.venv_hazm\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


 Fine-tuned SURf model is loaded and ready!


In [41]:
# @title Local Qwen2.5-VL Function

def get_qwen_response(query: str, image_path: str = None) -> str:
    """
    Generates a response using the locally loaded Qwen2.5-VL-7B-Instruct model.
    Handles both text-only and text-with-image prompts.
    """
    message_content = []
    image = None

    # Prepare the prompt content for the model
    if image_path:
        try:
            image = Image.open(image_path).convert("RGB")
            message_content.append({"type": "image", "image": image})
        except FileNotFoundError:
            print(f"Error: The image file was not found at {image_path}")
            query += f"\\n[Note: The user tried to upload an image at '{image_path}', but it was not found.]"

    # Add the text query to the content list
    message_content.append({"type": "text", "text": query})

    # Format the complete message structure for the chat model
    messages = [{"role": "user", "content": message_content}]

    # Apply the chat template to create the final prompt string
    text_prompt = qwen_processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Prepare inputs for the model
    inputs = qwen_processor(
        text=[text_prompt],
        images=[image] if image else None,
        return_tensors="pt"
    ).to(device)

    try:
        # Generate token IDs as a response from the model
        generated_ids = qwen_model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.7,
            top_p=0.95
        )

        # Decode the generated token IDs back into a text string
        response_text = qwen_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # The model's raw output includes the full prompt. The actual answer
        # starts after the "assistant\n" marker.
        assistant_marker = "assistant\n"
        if assistant_marker in response_text:
            # Split the string by the marker and take the last part, then strip whitespace
            cleaned_response = response_text.split(assistant_marker)[-1].strip()
        else:
            # If the marker isn't found for some reason, return the raw text as a fallback
            cleaned_response = response_text.strip()

        return cleaned_response


    except Exception as e:
        return f"An error occurred during model generation: {e}"

# For compatibility with your downstream cells, assign the new function to `text_generator`
text_generator = get_qwen_response



In [42]:

import base64

def base64_encode_image(image_path):
    """
    Takes an image path and returns its Base64 encoded string
    to embed it directly in the notebook.
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding image {image_path}: {e}")
        return ""

In [43]:

def generate_list_of_places(query: str, k: int = 3):
    """
    An enhanced RAG function that retrieves K documents, their names, descriptions,
    and a primary image for each, then formats them into a more detailed list.
    """
    logs = ["--- Running Enhanced List Generation RAG ---"]

    # 1. Retrieve the top K documents related to the query
    logs.append(f"Retrieving top {k} documents for query: '{query}'")
    query_embedding = text_retriever_model.encode([query])
    faiss.normalize_L2(query_embedding)
    distances, indices = text_only_faiss_index.search(query_embedding, k=k)

    if not np.any(indices[0]):
        return "متاسفانه مکانی مرتبط با سوال شما پیدا نکردم.", None, "\n".join(logs)

    # 2. Collect names, descriptions, and image paths for the retrieved places
    place_details = []
    retrieved_image_paths = [] # To store paths for display

    for rank, idx in enumerate(indices[0]):
        doc = tourism_data[idx]
        name = doc.get('name', 'N/A')
        description = doc.get('description', 'پاسخ دقیقی یافت نشد.')

        # Get one image for each place in the list
        image_path = get_image_path(doc, index=0) # Assuming get_image_path returns the first image
        if image_path:
            retrieved_image_paths.append(image_path)

        place_details.append({
            'name': name,
            'description': description,
            'image': image_path # Store image path with details
        })
        logs.append(f"Retrieved place {rank+1}: '{name}' (Image: {image_path})")

    # 3. Use the LLM to format the list with names and descriptions
    # We will pass the details as a structured string for the LLM to format
    formatted_details_for_llm = []
    for detail in place_details:
        formatted_details_for_llm.append(f"نام: {detail['name']}\nتوضیحات: {detail['description']}")

    list_prompt = (
        "You are a helpful and engaging tour guide assistant. Based on the following information for several places, "
        "present them to the user in a friendly, numbered list format in Persian. For each place, include its name "
        "and a concise summary of its description. Start with a welcoming sentence relevant to the user's query.\n\n"
        f"""User's original query: '{query}'
        Information about places:
        ---
        {'---'
        ''.join(formatted_details_for_llm)}
        ---
        
        
        Formatted response:"""
    )

    final_list_text = get_qwen_response(query=list_prompt)


    return final_list_text, retrieved_image_paths, "\n".join(logs)

In [44]:
# FLARE Helper Functions
import typing

def is_low_confidence(sentence: str, original_query: str) -> typing.Tuple[bool, str]:
    """
    Uses the LLM to check if a generated sentence is low-confidence.
    Returns a tuple: (is_uncertain_boolean, log_message_string).
    """
    prompt = (
        f"A user asked this question: \"{original_query}\"\n\n"
        f"In response, the system generated this statement: \"{sentence}\"\n\n"
        "Is the generated statement a specific, confident fact that directly helps answer the user's question, "
        "or is it a vague, uncertain guess, unrelated to user's question or a wrong answer? Answer with only the word **CONFIDENT** if the answer is reliable or **UNCERTAIN** if not."
    )
    response = get_qwen_response(query=prompt)

    # Create the log message instead of printing it
    log_message = f"Confidence Check on \"{sentence[:50]}...\": Model answered '{response}'"
    is_uncertain = "UNCERTAIN" in response.upper()

    return is_uncertain, log_message

def create_query_from_sentence(sentence: str) -> typing.Tuple[str, str]:
    """
    Uses the LLM to turn a sentence into a search query.
    Returns a tuple: (new_query_string, log_message_string).
    """
    prompt = (
        f"Extract the most important keywords or entities from the following sentence to use as a search query.\n\n"
        f"Sentence: \"{sentence}\"\n\n"
        "Search Keywords:"
    )
    question_query = get_qwen_response(query=prompt)

    # Create the log message instead of printing it
    log_message = f"Created new forward-looking query: \"{question_query}\""

    return question_query, log_message

In [45]:
# CORRECTED version of your original hybrid_rag_system function

def hybrid_rag_system(query: str, text_faiss_index, image_faiss_index, image_metadata, image_path: str = None):
    """
    Retrieves context from text or image indexes based on the query type.
    - If image_path is provided, it performs an image-based search.
    - Otherwise, it performs a text-based search.
    """
    context_text = ""
    retrieved_image_paths = []

    print(f"--- RAG Input ---")
    print(f"Query: {textwrap.shorten(query, width=100)}")
    if image_path:
        print(f"Image Path: {image_path}")
    print("-" * 20)
    
    # Check if this is an image query
    if image_path and os.path.exists(image_path):
        try:
            image = Image.open(image_path)
            image_inputs = clip_processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                query_embedding = clip_model.get_image_features(**image_inputs)
            query_embedding /= query_embedding.norm(p=2, dim=-1, keepdim=True)
            query_embedding_np = query_embedding.cpu().numpy().astype('float32')
            
            distances, indices = image_faiss_index.search(query_embedding_np, k=3)

            print("--- Retrieved Documents (from Image Search) ---")
            for i, idx in enumerate(indices[0]):
                loc_idx = image_metadata[idx]['loc_idx']
                location_data = tourism_data[loc_idx]
                retrieved_image_paths.append(image_metadata[idx]['path'])
                context_text += f"--- سند مرتبط شماره {i+1} ---\nنام مکان: {location_data.get('name', 'N/A')}\nتوضیحات: {location_data.get('description', 'N/A')}\n\n"
                print(f"Doc {i+1}: {location_data.get('name', 'N/A')} (Dist: {distances[0][i]:.4f})")
        except Exception as e:
            print(f"Error during image processing: {e}")
            
    # If not an image query, perform a text query
    else:
        query_embedding = text_retriever_model.encode(query, convert_to_tensor=False)
        query_embedding_np = np.array([query_embedding], dtype='float32')
        faiss.normalize_L2(query_embedding_np)
        
        distances, indices = text_faiss_index.search(query_embedding_np, k=3)
        
        print("--- Retrieved Documents (from Text Search) ---")
        for i, idx in enumerate(indices[0]):
            location_data = tourism_data[idx]
            image_paths = get_all_image_paths(location_data)
            if image_paths:
                retrieved_image_paths.append(image_paths[0])
            context_text += f"--- سند مرتبط شماره {i+1} ---\nنام مکان: {location_data.get('name', 'N/A')}\nتوضیحات: {location_data.get('description', 'N/A')}\n\n"
            print(f"Doc {i+1}: {location_data.get('name', 'N/A')} (Dist: {distances[0][i]:.4f})")

    final_image_path = retrieved_image_paths[0] if retrieved_image_paths else image_path
    
    print("-" * 20)
    display(HTML(f"<p style='font-family:monospace; white-space: pre-wrap;'><b>Retrieved Context:</b>\n{context_text}</p>"))
    
    return context_text, final_image_path

In [46]:
def answer_query_intelligently(query: str, image_path: str = None, text_faiss_index=None, image_faiss_index=None, image_metadata=None, text_retriever_model=None):
    """
     This function acts as a dispatcher.
    It first detects the user's intent and then calls the correct RAG system.
    """
    # If an image is provided, it's always a "deep dive" query.
    if image_path:
        print("--- Intent: Deep Dive (Image Provided) ---")
        return hybrid_rag_system(query=query, image_path=image_path, text_faiss_index=text_faiss_index, image_faiss_index=image_faiss_index, image_metadata=image_metadata, text_retriever_model=text_retriever_model)

    # If no image, ask the LLM to classify the text query
    intent_prompt = (
        "You are an expert query classifier. Classify the user's query into one of two categories: "
        "1. DEEP_DIVE: The user is asking about a specific place, person, or thing. "
        "   (Examples: 'Tell me about Shorabil Lake', 'Where is Shah Qasim Dam?', 'What are the features of Abadar Village?') "
        "2. LIST_REQUEST: The user is asking for a list of multiple places or things. "
        "   (Examples: 'What are some places to visit in Ardabil?', 'recommend tourist spots in Yasuj', 'list of historical sites')\n\n"
        +f"User query: \"{query}\"\n\n"+
        "Classification (respond with only DEEP_DIVE or LIST_REQUEST):"
    )

    intent = get_qwen_response(query=intent_prompt)
    print(f"--- Intent Detected: {intent} ---")

    if "DEEP_DIVE" in intent:
        # Use the powerful FLARE system for specific questions
        return hybrid_rag_system(query=query, text_faiss_index=text_faiss_index, image_faiss_index=image_faiss_index, image_metadata=image_metadata, text_retriever_model=text_retriever_model)
    elif "LIST_REQUEST" in intent:
        # Use the simple list generator for broad questions
        return generate_list_of_places(query=query)
    else:
        # A fallback in case the classification is unclear
        print("--- Intent unclear, defaulting to Deep Dive ---")
        return hybrid_rag_system(query=query, text_faiss_index=text_faiss_index, image_faiss_index=image_faiss_index, image_metadata=image_metadata, text_retriever_model=text_retriever_model)

In [47]:
import json
import os

# --- Load and Prepare Datasets ---
# Ensure your json files are in the PROJECT_PATH directory
exam_path_seen = os.path.join(PROJECT_PATH, "exam.json")
exam_path_unseen = os.path.join(PROJECT_PATH, "exam_unseen.json")

with open(exam_path_seen, "r", encoding="utf-8") as f:
    eval_data_seen = json.load(f)

with open(exam_path_unseen, "r", encoding="utf-8") as f:
    eval_data_unseen_full = json.load(f)

print(f"Loaded {len(eval_data_seen)} items from exam.json")
print(f"Loaded {len(eval_data_unseen_full)} items from exam_unseen.json")

# Create a set of questions from the seen data for efficient lookup
seen_questions = {item['question'] for item in eval_data_seen}

# Filter the unseen_full list to get only the new items
eval_data_unseen_only = [
    item for item in eval_data_unseen_full
    if item['question'] not in seen_questions
]

print(f"Found {len(eval_data_unseen_only)} new items in exam_unseen.json for evaluation.")

Loaded 80 items from exam.json
Loaded 100 items from exam_unseen.json
Found 20 new items in exam_unseen.json for evaluation.


In [48]:
import textwrap
import re
from IPython.display import display, HTML

# 1. A "Silent" Retriever: This version just gets the context without printing anything itself.
def hybrid_rag_system_silent(query: str, text_faiss_index, image_faiss_index, image_metadata, image_path: str = None):
    context_text = ""
    retrieved_image_paths = []
    
    if image_path and os.path.exists(image_path):
        # Image-based search logic
        image = Image.open(image_path)
        image_inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            query_embedding = clip_model.get_image_features(**image_inputs)
        query_embedding /= query_embedding.norm(p=2, dim=-1, keepdim=True)
        query_embedding_np = query_embedding.cpu().numpy().astype('float32')
        _, indices = image_faiss_index.search(query_embedding_np, k=3)
        for i, idx in enumerate(indices[0]):
            loc_idx = image_metadata[idx]['loc_idx']
            location_data = tourism_data[loc_idx]
            retrieved_image_paths.append(image_metadata[idx]['path'])
            context_text += f"--- سند مرتبط شماره {i+1} ---\nنام مکان: {location_data.get('name', 'N/A')}\nتوضیحات: {location_data.get('description', 'N/A')}\n\n"
    else:
        # Text-based search logic
        query_embedding = text_retriever_model.encode(query, convert_to_tensor=False)
        query_embedding_np = np.array([query_embedding], dtype='float32')
        faiss.normalize_L2(query_embedding_np)
        _, indices = text_faiss_index.search(query_embedding_np, k=3)
        for i, idx in enumerate(indices[0]):
            location_data = tourism_data[idx]
            image_paths = get_all_image_paths(location_data)
            if image_paths:
                retrieved_image_paths.append(image_paths[0])
            context_text += f"--- سند مرتبط شماره {i+1} ---\nنام مکان: {location_data.get('name', 'N/A')}\nتوضیحات: {location_data.get('description', 'N/A')}\n\n"

    final_image_path = retrieved_image_paths[0] if retrieved_image_paths else image_path
    return context_text, final_image_path

# 2. The New HTML Interface Functions
def display_retrieval_interface(q_num, total_q, question, options, context):
    # Process context for display
    docs = context.strip().split('--- سند مرتبط شماره ')
    docs_html = ""
    for doc_text in docs:
        if not doc_text: continue
        doc_num = doc_text[0]
        content = doc_text[5:].strip().replace('نام مکان: ', '<b>نام مکان:</b> ').replace('توضیحات: ', '<br><b>توضیحات:</b> ')
        docs_html += f"""
        <div style="border: 1px solid #e0e0e0; padding: 10px; margin-top: 10px; border-radius: 5px;">
            <h4>سند مرتبط شماره {doc_num}</h4>
            <p style="white-space: pre-wrap;">{content}</p>
        </div>
        """

    html = f"""
    <div style="border: 2px solid #4a90e2; padding: 20px; font-family: sans-serif; border-radius: 8px; margin-bottom: 20px;">
        <h2 style="color: #4a90e2; border-bottom: 1px solid #ddd; padding-bottom: 10px;">
            ارزیابی سوال {q_num} از {total_q}
        </h2>
        
        <h3 style="color: #333;">سوال:</h3>
        <p style="font-size: 1.1em; background-color: #f7f7f7; padding: 15px; border-radius: 5px;">{question}</p>
        
        <h3 style="color: #333;">گزینه‌ها:</h3>
        <ol>{''.join([f'<li>{opt}</li>' for opt in options])}</ol>
        
        <h3 style="color: #333; margin-top: 25px; border-top: 1px solid #ddd; padding-top: 15px;">اسناد بازیابی شده:</h3>
        {docs_html}
    </div>
    """
    display(HTML(html))

def display_generation_results(model_answer, correct_answer):
    is_correct = model_answer == correct_answer
    border_color = "#28a745" if is_correct else "#dc3545"
    result_text = "صحیح" if is_correct else "غلط"

    html = f"""
    <div style="border: 2px solid {border_color}; padding: 15px; font-family: sans-serif; border-radius: 8px; margin-top:10px;">
        <h3 style="margin-top:0;">نتیجه تولید پاسخ: <span style='color:{border_color};'>{result_text}</span></h3>
        <p><b>پاسخ مدل:</b> {model_answer}</p>
        <p><b>پاسخ صحیح:</b> {correct_answer}</p>
    </div>
    """
    display(HTML(html))

# 3. The Updated Evaluation Loop (using the interface)
# This function for getting input remains the same
def get_user_input_for_evaluation():
    # ... [The get_user_input_for_evaluation function from the previous turn goes here, unchanged]
    while True:
        try:
            num_retrieved = int(input("1. How many of the 3 retrieved documents are RELEVANT? (0, 1, 2, or 3): "))
            if num_retrieved not in [0, 1, 2, 3]: raise ValueError
            break
        except ValueError: print("Invalid input. Please enter a number between 0 and 3.")
    while True:
        hit_2_document = input("2. Is the answer in the first TWO retrieved documents? (y/n): ").lower()
        if hit_2_document in ['y', 'n']: break
        else: print("Invalid input. Please enter 'y' or 'n'.")
    if hit_2_document == 'y':
        is_answer_in_documents = 'y'
    else:
        while True:
            is_answer_in_documents = input("3. Is the answer in ANY of the retrieved documents? (y/n): ").lower()
            if is_answer_in_documents in ['y', 'n']: break
            else: print("Invalid input. Please enter 'y' or 'n'.")
    return num_retrieved, hit_2_document == 'y', is_answer_in_documents == 'y'

def run_rag_test_with_evaluation(q_num, total_q, question, options, correct_answer, text_faiss_index, image_faiss_index, image_metadata):
    # Step 1: Retrieve context silently
    context, image_for_prompt = hybrid_rag_system_silent(question, text_faiss_index, image_faiss_index, image_metadata)
    
    # Step 2: Display the HTML interface for the retriever
    display_retrieval_interface(q_num, total_q, question, options, context)
    
    # Step 3: Get human feedback for the retriever
    num_retrieved, hit_2, has_answer = get_user_input_for_evaluation()
    
    # Step 4: Generate a response
    prompt = (f"شما یک دستیار هوشمند و دقیق هستید...\n" # Using your original full prompt here
              f"--- اطلاعات زمینه ---\n{context}\n\n"
              f"--- سوال کاربر ---\n{question}\n\n"
              f"--- گزینه‌ها ---\n{', '.join(options)}\n\n"
              f"--- پاسخ شما (دقیقاً یکی از گزینه‌ها) ---\n")
    response = get_qwen_response(query=prompt, image_path=image_for_prompt)
    
    # Step 5: Display the results interface
    display_generation_results(response, correct_answer)

    # Step 6: Get human feedback for the generator
    while True:
        g_answer = input("Was the GENERATED answer correct? (y/n): ").lower()
        if g_answer in ['y', 'n']: break
        else: print("Invalid input. Please enter 'y' or 'n'.")

    return response == correct_answer, num_retrieved, has_answer, hit_2, g_answer == 'y'

def evaluate_rag_model(eval_data, text_faiss_index, image_faiss_index, image_metadata):
    # ... [The evaluate_rag_model function from the previous turn goes here, with one small change]
    # Find this line:
    # auto_correct, num_ret, has_ans, hit_2, g_ans = run_rag_test_with_evaluation(...)
    # And add q_num and total_q to it like this:
    # auto_correct, num_ret, has_ans, hit_2, g_ans = run_rag_test_with_evaluation(i + 1, len(eval_data), ...)
    
    total_num_retrieved = 0
    has_answer_count = 0
    hit_2_count = 0
    g_answer_correct_count = 0
    automated_correct_count = 0

    for i, entry in enumerate(eval_data):
        auto_correct, num_ret, has_ans, hit_2, g_ans = run_rag_test_with_evaluation(
            i + 1, len(eval_data), # <-- The only change is here
            entry['question'], entry['options'], entry['correct_answer'], 
            text_faiss_index, image_faiss_index, image_metadata
        )
        total_num_retrieved += num_ret
        if has_ans: has_answer_count += 1
        if hit_2: hit_2_count += 1
        if g_ans: g_answer_correct_count += 1
        if auto_correct: automated_correct_count += 1
            
    num_questions = len(eval_data)
    final_results = {
        "num_questions": num_questions, "total_num_retrieved": total_num_retrieved,
        "has_answer_count": has_answer_count, "hit_2_count": hit_2_count,
        "g_answer_correct_count": g_answer_correct_count,
    }
    return final_results

In [49]:
# --- Execute Evaluation on SEEN Data ---
print("====================================================================")
print("===> STARTING HUMAN-IN-THE-LOOP EVALUATION ON SEEN DATA (from exam.json)")
print("====================================================================")
seen_results = evaluate_rag_model(
    eval_data_seen,
    text_only_faiss_index,
    image_only_faiss_index,
    image_metadata
)

# --- Execute Evaluation on UNSEEN Data ---
print("\n\n====================================================================")
print("===> STARTING HUMAN-IN-THE-LOOP EVALUATION ON UNSEEN DATA (new items)")
print("====================================================================")
unseen_results = evaluate_rag_model(
    eval_data_unseen_only,
    text_only_faiss_index,
    image_only_faiss_index,
    image_metadata
)

===> STARTING HUMAN-IN-THE-LOOP EVALUATION ON SEEN DATA (from exam.json)


Error: The image file was not found at E:\project\pasta\NLP_Project\Data\i


Error: The image file was not found at E:\project\pasta\NLP_Project\Data\i


Error: The image file was not found at E:\project\pasta\NLP_Project\Data\i


Error: The image file was not found at E:\project\pasta\NLP_Project\Data\i


...

Invalid input. Please enter a number between 0 and 3.


Error: The image file was not found at E:\project\pasta\NLP_Project\Data\i




===> STARTING HUMAN-IN-THE-LOOP EVALUATION ON UNSEEN DATA (new items)


Error: The image file was not found at E:\project\pasta\NLP_Project\Data\i


Error: The image file was not found at E:\project\pasta\NLP_Project\Data\i


In [ ]:
# --- Calculate and Display Metrics for SEEN Data ---
print("\n\n====================== RESULTS FOR SEEN DATA ======================")
num_q_seen = seen_results["num_questions"]
if num_q_seen > 0:
    # Precision@3: How relevant were the retrieved documents?
    precision_at_3_seen = (seen_results["total_num_retrieved"] / (num_q_seen * 3)) * 100
    print(f"Retriever Precision@3: {precision_at_3_seen:.2f}%")

    # Answer Recall Rate: Did the retriever find the necessary info?
    recall_rate_seen = (seen_results["has_answer_count"] / num_q_seen) * 100
    print(f"Retriever Answer Recall Rate: {recall_rate_seen:.2f}%")

    # Hit Rate@2: Was the answer ranked highly by the retriever?
    hit_rate_2_seen = (seen_results["hit_2_count"] / num_q_seen) * 100
    print(f"Retriever Hit Rate@2: {hit_rate_2_seen:.2f}%")

    # Generator Accuracy: Was the final answer correct (based on human judgment)?
    generator_accuracy_seen = (seen_results["g_answer_correct_count"] / num_q_seen) * 100
    print(f"Generator End-to-End Accuracy: {generator_accuracy_seen:.2f}%")
else:
    print("No seen data was evaluated.")

# --- Calculate and Display Metrics for UNSEEN Data ---
print("\n====================== RESULTS FOR UNSEEN DATA ======================")
num_q_unseen = unseen_results["num_questions"]
if num_q_unseen > 0:
    # Precision@3
    precision_at_3_unseen = (unseen_results["total_num_retrieved"] / (num_q_unseen * 3)) * 100
    print(f"Retriever Precision@3: {precision_at_3_unseen:.2f}%")

    # Answer Recall Rate
    recall_rate_unseen = (unseen_results["has_answer_count"] / num_q_unseen) * 100
    print(f"Retriever Answer Recall Rate: {recall_rate_unseen:.2f}%")

    # Hit Rate@2
    hit_rate_2_unseen = (unseen_results["hit_2_count"] / num_q_unseen) * 100
    print(f"Retriever Hit Rate@2: {hit_rate_2_unseen:.2f}%")
    
    # Generator Accuracy
    generator_accuracy_unseen = (unseen_results["g_answer_correct_count"] / num_q_unseen) * 100
    print(f"Generator End-to-End Accuracy: {generator_accuracy_unseen:.2f}%")
else:
    print("No unseen data was evaluated.")
    
print("\n======================================================================\n")



====================== RESULTS FOR SEEN DATA ======================
Retriever Precision@3: 55.42%
Retriever Answer Recall Rate: 85.00%
Retriever Hit Rate@2: 82.50%
Generator End-to-End Accuracy: 81.25%

====================== RESULTS FOR UNSEEN DATA ======================
Retriever Precision@3: 51.67%
Retriever Answer Recall Rate: 80.00%
Retriever Hit Rate@2: 80.00%
Generator End-to-End Accuracy: 80.00%




In [51]:
import numpy as np
import textwrap

# This is a streamlined version of your RAG system for silent, automated execution.
def hybrid_rag_system_automated(query: str, text_faiss_index, image_faiss_index, image_metadata, image_path: str = None):
    context_text = ""
    retrieved_image_paths = []

    if image_path and os.path.exists(image_path):
        # This part is for multimodal queries, which are not in your text-only exam files.
        # It's kept here for completeness if you ever create a multimodal exam file.
        image = Image.open(image_path)
        image_inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            query_embedding = clip_model.get_image_features(**image_inputs)
        query_embedding /= query_embedding.norm(p=2, dim=-1, keepdim=True)
        query_embedding_np = query_embedding.cpu().numpy().astype('float32')
        _, indices = image_faiss_index.search(query_embedding_np, k=3)

        for i, idx in enumerate(indices[0]):
            loc_idx = image_metadata[idx]['loc_idx']
            location_data = tourism_data[loc_idx]
            retrieved_image_paths.append(image_metadata[idx]['path'])
            context_text += f"--- سند مرتبط شماره {i+1} ---\nنام مکان: {location_data.get('name', 'N/A')}\nتوضیحات: {location_data.get('description', 'N/A')}\n\n"
    else:
        # Text Query for your exam files
        query_embedding = text_retriever_model.encode(query, convert_to_tensor=False)
        query_embedding_np = np.array([query_embedding], dtype='float32')
        faiss.normalize_L2(query_embedding_np)
        _, indices = text_faiss_index.search(query_embedding_np, k=3)

        for i, idx in enumerate(indices[0]):
            location_data = tourism_data[idx]
            image_paths = get_all_image_paths(location_data)
            if image_paths:
                retrieved_image_paths.append(image_paths[0])
            context_text += f"--- سند مرتبط شماره {i+1} ---\nنام مکان: {location_data.get('name', 'N/A')}\nتوضیحات: {location_data.get('description', 'N/A')}\n\n"

    final_image_path = retrieved_image_paths[0] if retrieved_image_paths else image_path
    return context_text, final_image_path

# This is the main automated evaluation loop.
def run_automated_evaluation(eval_data, text_faiss_index, image_faiss_index, image_metadata):
    correct_answers = 0
    total_questions = len(eval_data)
    
    print(f"Starting automated evaluation for {total_questions} items...")

    for i, entry in enumerate(eval_data):
        question = entry['question']
        options = entry['options']
        correct_answer = entry['correct_answer']

        context, image_for_prompt = hybrid_rag_system_automated(
            query=question,
            text_faiss_index=text_faiss_index,
            image_faiss_index=image_faiss_index,
            image_metadata=image_metadata
        )

        prompt = (
            "شما یک دستیار هوشمند و دقیق هستید. وظیفه شما پاسخ به سوال کاربر فقط و فقط بر اساس اطلاعات موجود در \"سندهای مرتبط\" است. "
            "شما باید از بین گزینه‌های ارائه شده، یک پاسخ صحیح انتخاب کنید. پاسخ شما باید دقیقاً یکی از این گزینه‌ها باشد.\n\n"
            f"--- اطلاعات زمینه ---\n{context}\n\n"
            f"--- سوال کاربر ---\n{question}\n\n"
            f"--- گزینه‌ها ---\n{', '.join(options)}\n\n"
            "--- پاسخ شما (دقیقاً یکی از گزینه‌ها) ---\n"
        )

        response = get_qwen_response(query=prompt, image_path=image_for_prompt)
        
        cleaned_response = None
        for opt in options:
            if opt in response:
                cleaned_response = opt
                break
        
        print(f"Question {i+1}/{total_questions}: {question[:80]}...")
        print(f"Model Answer: '{cleaned_response}' | Correct Answer: '{correct_answer}'")

        if cleaned_response == correct_answer:
            correct_answers += 1
            print("Result: CORRECT\n")
        else:
            print("Result: INCORRECT\n")

    accuracy = (correct_answers / total_questions) * 100 if total_questions > 0 else 0
    return accuracy

In [52]:
# --- Run Evaluation on SEEN Data ---
print("====================================================================")
print("===> STARTING EVALUATION ON SEEN DATA (from exam.json)")
print("====================================================================")
seen_accuracy = run_automated_evaluation(
    eval_data_seen,
    text_only_faiss_index,
    image_only_faiss_index,
    image_metadata
)
print("-" * 50)

# --- Run Evaluation on UNSEEN Data ---
print("\n====================================================================")
print("===> STARTING EVALUATION ON UNSEEN DATA (new items from exam_unseen.json)")
print("====================================================================")
unseen_accuracy = run_automated_evaluation(
    eval_data_unseen_only,
    text_only_faiss_index,
    image_only_faiss_index,
    image_metadata
)
print("-" * 50)

# --- Final Summary ---
print("\n======================== AUTOMATED EVALUATION SUMMARY ========================")
print(f"End-to-End Accuracy on SEEN data (exam.json): {seen_accuracy:.2f}%")
print(f"End-to-End Accuracy on UNSEEN data (new items): {unseen_accuracy:.2f}%")
print("==========================================================================\n")

===> STARTING EVALUATION ON SEEN DATA (from exam.json)
Starting automated evaluation for 80 items...
Question 1/80: چشمه‌های آبگرم سرعین در استان اردبیل به خاطر چه ویژگی‌ای شهرت دارند؟...
Model Answer: 'آب‌های معدنی با خواص درمانی' | Correct Answer: 'آب‌های معدنی با خواص درمانی'
Result: CORRECT

Question 2/80: کدام یک از مکان‌های زیر در استان اردبیل به عنوان جاذبه تاریخی شناخته می‌شود؟...
Model Answer: 'None' | Correct Answer: 'قلعه ارشق ارشک'
Result: INCORRECT

Error: The image file was not found at E:\project\pasta\NLP_Project\Data\i
Question 3/80: کدام دریاچه در استان اردبیل در مرز استان‌های گیلان و اردبیل قرار دارد؟...
Model Answer: 'دریاچه سوها' | Correct Answer: 'دریاچه نئور'
Result: INCORRECT

Question 4/80: کدام یک از جاذبه‌های طبیعی اردبیل در نزدیکی شهرستان مشگین‌شهر واقع شده است؟...
Model Answer: 'چشمه‌های آبگرم سرعین' | Correct Answer: 'آبشار کرکری'
Result: INCORRECT

Question 5/80: کدام مکان در استان اردبیل یکی از قدیمی‌ترین مساجد ایران است؟...
Model Answer: 'مسجد شیخ صفی' 

In [ ]:

import gradio as gr
from PIL import Image
import os # Make sure os is imported if not already

# ==============================================================================
# CORRECTED GRADIO INTERFACE CELL (Final Version)
# ==============================================================================

def gradio_interface(text_query, image_input):
    """
    Wrapper function for the new Gradio interface.
    It handles both text and image inputs intelligently.
    """
    if not text_query and image_input is None:
        return "لطفا یک سوال متنی بنویسید یا یک تصویر آپلود کنید.", None

    # *** THE FIX IS HERE ***
    # The 'image_input' from Gradio with type='filepath' IS the path string.
    # We no longer need to use .name
    image_path = image_input if image_input is not None else None

    # If there's no text query but an image is uploaded, use a default query.
    if not text_query and image_path:
        text_query = "این تصویر مربوط به کجاست و چه ویژگی‌هایی دارد؟"

    print("--- Calling RAG system from Gradio ---")

    # Call the final, correct RAG function
    response, retrieved_image_path = hybrid_rag_system(
        query=text_query,
        text_faiss_index=text_only_faiss_index,
        image_faiss_index=image_only_faiss_index,
        image_metadata=image_metadata,
        image_path=image_path
    )

    # Gradio needs a valid image path or None to display the output
    final_image_to_display = retrieved_image_path if retrieved_image_path and os.path.exists(retrieved_image_path) else None

    return response, final_image_to_display

# Create a modern, multimodal Gradio interface
demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="سوال خود را وارد کنید (اختیاری اگر تصویر آپلود می‌کنید)"),
        # *** THE FIX IS HERE ***
        # Changed type="file" to type="filepath"
        gr.Image(type="filepath", label="برای پرسش، یک تصویر آپلود کنید (اختیاری)")
    ],
    outputs=[
        gr.Textbox(label="پاسخ تولید شده"),
        gr.Image(type="filepath", label="تصویر مرتبط")
    ],
    title="🤖 سامانه پرسش و پاسخ گردشگری هوشمند",
    description="سوال خود را بنویسید یا یک عکس از مکان‌های دیدنی ایران آپلود کنید تا اطلاعات مربوط به آن را دریافت کنید.",
    examples=[
        ["قلعه پرتغالی‌ها در کدام جزیره است؟", None],
        ["اینجا کجاست؟", "E:/project/pasta/NLP_Project/Data/images/Ardabil_imgs/Abadar_Rock_Village_1.jpg"]
    ]
)

# Launch the demo
print("Launching Gradio demo...")
demo.launch(share=True, debug=True)